# Horn-satisfiability
Yesterday I spent some time on [unification](https://en.wikipedia.org/wiki/Unification_%28computer_science%29) which is a building block of [Prolog](https://en.wikipedia.org/wiki/Prolog) programs. Another important concept used in Prolog is satisfiability of propositional formulae.

In general, propositional formulae satisfiability, or SAT problem, is NP-complete and therefore provably intractable.

Fortunately, there is a special case called [Horn satisfiability](https://en.wikipedia.org/wiki/Horn-satisfiability) which can be solved in linear time proportionally to number of occurrences of negative literals.

Horn clause has a form `A & B & C … => P`, or equivalently can be written as `¬A ∨ ¬B ∨ ¬C ∨ … ∨ P`. Hence the formula in disjunctive form contains a finite set of negative literals and at most one positive literal. We usually recognize three kinds of formulae.

* rule: `Q => P`
* fact: `true => P`
* goal: `P => false`

Function `conc` I implemented yesterday can be written in logic like this.

```
X=.() & Y=Z => conc(X, Y, Z)
X=.(H, T) & Z=.(H, S) & conc(T, S, Z) => conc(X, Y, Z)
```

To “call” this program we ask for satisfiability of this goal.

```
X=.(1, .()) & Y=.(2, .()) & conc(X, Y, Z) => false
```

And Prolog interpret will try to solve the problem and find the answer using proof by contradiction.

In [1]:
from collections import defaultdict, deque

## algorithm

In [2]:
def knowledge_base(formulas):
    rules, variable, dependency = [], defaultdict(bool), defaultdict(list)

    def _clause(formula):
        # A, B, C => P
        neg, pos = formula.replace(' ', '').split('=>')
        neg, pos = set(neg.split('&')) - {''}, pos or None

        # add rule
        rules.append((neg, pos))
        
        # set variable and track dependencies
        for i in neg:
            dependency[i].append((neg, pos))

    # parse formulas and build knowledge base
    deque((_clause(i) for i in formulas.split('\n') if i), 0)
    
    return rules, variable, dependency

In [3]:
def resolution(rules, variable, dependency):
    # initial variables that have to be satisfied
    to_satisfy = [(neg, pos) for neg, pos in rules if not neg]

    while to_satisfy:
        neg, pos = to_satisfy.pop()

        # contradiction: true => false
        if not pos:
            return False

        # satisfy variable
        variable[pos] = True

        # update dependent rules
        for d_neg, d_pos in dependency[pos]:
            d_neg.remove(pos)
            
            # next variable to be satisfied
            if not d_neg and d_pos not in variable:
                to_satisfy.append((d_neg, d_pos))

    return True

In [4]:
def hornsat(formulas):
    rules, variable, dependency = knowledge_base(formulas)
    satisfiable = resolution(rules, variable, dependency)

    print(['CONTRADICTION', 'SATISFIABLE'][satisfiable])
    print(', '.join('%s=%s' % i for i in variable.items()))

## run

In [5]:
hornsat("""
X => Y
Y => Z
=> X
""")

SATISFIABLE
X=True, Y=True, Z=True


In [6]:
hornsat("""
X => Y
Y => X
=> X
Y =>
""")

CONTRADICTION
X=True, Y=True


In [7]:
hornsat("""
P & Q & R & S => X
P & Q => R
R => S
X =>
=> P
=> R
""")

SATISFIABLE
R=True, S=True, P=True


In [8]:
hornsat("""
P & Q & R & S => X
P & Q => R
R => S
X =>
=> P
=> Q
""")

CONTRADICTION
Q=True, P=True, R=True, S=True, X=True
